In [1]:
import sys
sys.path.append('..')
import pythonmatsim.api.logging
import pythonmatsim.api.jvmconfig as jc

In [ ]:
import jpype
import jpype.imports as jimport

jimport.registerDomain('org')

try:
    jc.build_and_start_jvm()
except jpype.JavaException as exception:
    print(exception.message())
    print(exception.stacktrace())
    

2019-05-08 14:17:44,183 - pythonmatsim.api.jvmconfig - DEBUG - generating classpath in /tmp/tmpk36ogktc


In [1]:
from pythonmatsim.api.events import *

ModuleNotFoundError: No module named 'pythonmatsim'

In [4]:


from org.matsim.core.scenario import ScenarioUtils
from org.matsim.core.config import ConfigUtils
from org.matsim.api.core.v01 import Id
from org.matsim.core.controler import Controler
from org.matsim.api.core.v01.events.handler import ActivityStartEventHandler, ActivityEndEventHandler
# Note: this does not work when using JPackage approach
from org.matsim.core.controler.OutputDirectoryHierarchy import OverwriteFileSetting


In [5]:
from java.net import URL, URI
from collections import defaultdict
import time

In [6]:
config_url = URI('https://raw.githubusercontent.com/matsim-org/matsim/master/examples/scenarios/siouxfalls-2014/config_default.xml').toURL()
#config_url = URI('https://raw.githubusercontent.com/matsim-org/matsim/master/examples/scenarios/equil/config.xml').toURL()
config = ConfigUtils.loadConfig(config_url)
config.controler().setLastIteration(1)
config.controler().setOverwriteFileSetting(OverwriteFileSetting.deleteDirectoryIfExists)


In [7]:
scenario = ScenarioUtils.loadScenario(config)
controler = Controler(scenario)

start = time.time()
controler.run()
duration_without_events = time.time() - start
duration_without_events

77.74684453010559

In [8]:
class TestListener(EventListener):
    def __init__(self):
        self.counts = defaultdict(int)

    def reset(self, iteration):
        self.iteration = iteration

    @listen_to(EventType.actStart, EventType.actEnd)
    def handleAct(self, event):
        event.time
        event.linkId
        event.facilityId
        event.persId
        event.actType
        self.counts[self.iteration] += 1

In [9]:
listener = TestListener()

In [10]:
scenario = ScenarioUtils.loadScenario(config)
controler = Controler(scenario)
add_event_handler(
    controler,
    listener,
    10)

start = time.time()
controler.run()
duration_with_events = time.time() - start
duration_with_events

2019-05-07 13:10:51,410 - pythonmatsim.api.events - INFO - Constructing listener list
2019-05-07 13:10:51,411 - pythonmatsim.api.events - INFO - Method handleAct listens to the following event types: {'actStart', 'actEnd'}


105.7708170413971

In [11]:
listener.counts


defaultdict(int, {0: 168537, 1: 173537})

In [12]:
class NaiveHandler:
    ####################################################################
    # Methods for the python side
    ####################################################################
    def __init__(self):
        self.counts = defaultdict(int)

    ####################################################################
    # Methods for the java side
    ####################################################################
        
    def handleEvent(self, event):
        event.getTime()
        event.getLinkId()
        event.getFacilityId()
        event.getPersonId()
        event.getActType()
        self.counts[self.iteration] += 1
        
    def reset(self, iteration):
        self.iteration = iteration
    
    # JProxy does not provide default implementation for those methods:
    # provide one, to avoid crashes.
    # In true python way, the JProxy checks if a method exists only if actually
    # invoked.
    def hashCode(self):
        return 42
    
    def toString(self):
        return "%s" % self

handler = NaiveHandler()
handlerProxy = jp.JProxy((ActivityStartEventHandler, ActivityEndEventHandler), inst=handler)

In [13]:
controler = Controler(scenario)

controler.getEvents().addHandler(handlerProxy)

start = time.time()
controler.run()
duration_naive = time.time() - start
duration_naive

114.32495188713074